# Amazon Web Services

In [1]:
import os
import sys

# Supprimer les commentaires pour installer (requirements.txt)

# À installer dans tous les cas pour Google Colab et Github
if ('google.colab' in sys.modules) or ('CI' in os.environ):
    !pip3 install -q -U boto3

## Comprehend

In [2]:
import boto3

comprehend = boto3.client('comprehend')
text = 'J\'aime aller au cinéma' 
sentiment = comprehend.detect_sentiment(Text=text, LanguageCode='fr')
print(f"Text sentiment: {sentiment['Sentiment']}")

Text sentiment: POSITIVE
